# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [7]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [8]:
# your code here
from sqlalchemy import create_engine
driver   = 'mysql+pymysql:'
user     = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136'
database = 'orders'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
orders = pd.read_sql('SELECT * FROM orders', engine)
orders.head()



,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [9]:
# your code here

# Aggregate amount spent by unique customers
customersaggreg = orders.groupby('CustomerID').agg({'amount_spent':'sum'})
customersaggreg

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [10]:
# Divide customers into spending categories
q_75 = customersaggreg['amount_spent'].quantile(0.75)
q_95 = customersaggreg['amount_spent'].quantile(0.95)

condlist = [customersaggreg['amount_spent'] > q_95, customersaggreg['amount_spent'] > q_75]
choicelist = ['VIP', 'Preferred']
customersaggreg['customer_category'] = np.select(condlist, choicelist, default= 'Normal')
customersaggreg

,amount_spent,customer_category
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Normal
...,...,...
18280,180.60,Normal
18281,80.82,Normal
18282,178.05,Normal


In [11]:
# Select VIP customers
vip_customers = customersaggreg.loc[customersaggreg['customer_category'] == 'VIP'].sort_values(by='amount_spent', ascending=False)

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [14]:
# Create countries dataframe
country_customer_id = orders[['CustomerID','Country']].groupby(['CustomerID','Country']).count().reset_index()
country_customer_id

,CustomerID,Country
0,12346,United Kingdom
1,12347,Iceland
2,12348,Finland
3,12349,Italy
4,12350,Norway
...,...,...
4342,18280,United Kingdom
4343,18281,United Kingdom
4344,18282,United Kingdom
4345,18283,United Kingdom


In [15]:
# merge VIP dataframe with countries dataframe

vip_with_country = pd.merge(country_customer_id, vip_customers, on="CustomerID")

# group the dataframe by country and count the individual Customer IDs

vip_with_country.groupby("Country").agg({"CustomerID":"count"}).sort_values(by="CustomerID", ascending=False).head(5)

,CustomerID
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
Spain,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [17]:
# your code here
preferred_customers = customersaggreg.loc[customersaggreg['customer_category'] == 'Preferred'].sort_values(by='amount_spent', ascending=False)

In [18]:
# merge VIP and preferred Dataframes
vip_preferred = pd.merge(vip_customers, preferred_customers, on='CustomerID', axis=1)
vip_preferred

TypeError: merge() got an unexpected keyword argument 'axis'

In [19]:
# merge this dataframe with the country dataframe
vp_country = pd.merge(country_customer_id, vip_preferred, on="CustomerID")

# group the dataframe by country and count the individual Customer IDs
vp_country

NameError: name 'vip_preferred' is not defined

In [20]:
vp_country.groupby("Country").agg({"CustomerID":"count"}).sort_values(by="CustomerID", ascending=False).head(5)

NameError: name 'vp_country' is not defined